In [1]:
# test example
pred=['41401', '4139', '25000', '4019', '2720', '53081', '412']
ref=['41401', '41402', '4111', '9992', '25000', '4019', '2724', '53081', '412']

In [34]:
metrics.accuracy_score(pred, ref)

ValueError: Found input variables with inconsistent numbers of samples: [7, 9]

In [14]:
import pandas as pd
import pickle as pkl

AttributeError: module 'huggingface_hub' has no attribute 'hf_api'

In [8]:
# old accuracy
result={}
c=0
curr_header='icd9'
result[f'{c}_{curr_header}']={'ele_match':0,'ele_total':0}
#can't find <row> separator
result['<row>_error']=0
#unequal number of columns
result['<col>_mismatch']=0
result

def cel_match(mode,curr_header,cel_pred,cel_ref,result):
    for c in mode:
        result[f'{c}_{curr_header}']['ele_total']+=len(cel_ref)
    #iterate thru each element in a cell
    for ele_pred, ele_ref in zip(cel_pred, cel_ref):
        char_right=0 # counts number of chars matching
        char_len=len(ele_ref) # counts number of charcters in this column cell
        for c,d in zip(ele_pred,ele_ref): #c and d are each char in word a,b
            if c==d: char_right+=1 #if c,d match, count as char_right
        #find the length of the longer element/word
        max_len=max(len(ele_pred),len(ele_ref))
        #find char_wrong
        char_wrong=max_len-char_right
        #--crucial: different config modes:
        for c in mode:
            perc=c/100
            if not (perc>=0 and perc<=1): raise ValueError(f"Invalid config name for ColMatch: {c}. Please input a valid number for percentage between 0 and 100 inclusice") 
            #if number of matching chars smaller than length of word
            if char_wrong/char_len<=perc: 
                result[f'{c}_{curr_header}']['ele_match']+=1
#                 # --test
#                 print('pred: ',cel_pred)
#                 print('ref: ',cel_ref)
#                 return
    return result

mode=[0]
# for row_pred,row_ref in zip(old_pred, old_ref):
# cel_ref=row_ref.split(' <CEL> ')
# cel_pred=row_pred.split(' <CEL> ')
#call cel_match helper function
result=cel_match(mode=mode,curr_header=curr_header,cel_pred=pred,cel_ref=ref,result=result)
print(result)
result['0_icd9']['ele_match']/result['0_icd9'][ 'ele_total']*100 
    

{'0_icd9': {'ele_match': 1, 'ele_total': 9}, '<row>_error': 0, '<col>_mismatch': 0}


11.11111111111111

In [ ]:
# use new method

In [10]:

pred=['41401', '4139', '25000', '4019', '2720', '53081', '412']
ref=['41401', '41402', '4111', '9992', '25000', '4019', '2724', '53081', '412']

In [20]:
class_file=os.path.join('class_files','diag_icd_classes.pkl')
with open(class_file,'rb') as f:
    classes=pkl.load(f)

In [21]:
from sklearn import metrics
import pandas as pd

In [27]:
pred_df=pd.Series(pred).str.get_dummies()
pred_df

,25000,2720,4019,412,4139,41401,53081
0,0,0,0,0,0,1,0
1,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0
3,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0
5,0,0,0,0,0,0,1
6,0,0,0,1,0,0,0


In [30]:
big_pred=pred_df.reindex(columns=classes).fillna(0)
big_pred

,0030,0031,0038,0039,0041,0048,0049,0051,00581,0059,...,V8745,V8801,V8811,V8812,V8821,V9010,V9039,V9081,V9089,V9103
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# ref
ref_df=pd.Series(ref).str.get_dummies()
ref_df

,25000,2724,4019,4111,412,41401,41402,53081,9992
0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0
6,0,1,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0
8,0,0,0,0,1,0,0,0,0


In [31]:
big_ref=ref_df.reindex(columns=classes).fillna(0)
big_ref

,0030,0031,0038,0039,0041,0048,0049,0051,00581,0059,...,V8745,V8801,V8811,V8812,V8821,V9010,V9039,V9081,V9089,V9103
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
metrics.accuracy_score(big_pred, big_ref)

ValueError: Found input variables with inconsistent numbers of samples: [7, 9]

In [ ]:
dum_ref.loc[0][dum_pred.loc[0]!=dum_ref.loc[0]]